In [1]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, BatchNormalization, LocallyConnected2D, SeparableConv2D
import tensorflow.keras.optimizers as optimizers
from tensorflow.keras.constraints import max_norm
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import activations

In [2]:
from create_data import create_all_x_y

In [3]:
X, y = create_all_x_y()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape)
print(y_train.shape)

4 parties rejetées par le goban, reste 2474 parties
(15833, 9, 9, 49)
(15833, 82)


In [7]:
def model_julien():
    λ = optimizers.schedules.InverseTimeDecay( # learning rate
        initial_learning_rate=1e-3,
        decay_steps=15000,
        decay_rate=0.5)

    α = 0.4 # dropout rate

    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(9,9,49), padding='same'))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dropout(rate=α))
    model.add(Dense(648, activation='relu'))
    model.add(Dropout(rate=α))
    model.add(Dense(324, activation='relu'))
    model.add(Dense(162, activation='relu'))
    model.add(Dense(82, activation='relu'))
    

    model.compile(optimizers.RMSprop(λ), loss='mean_absolute_error')

    # optimizer : optimizers.RMSprop(λ) ; optimizers.Adam(learning_rate=λ) ; optimizers.SGD(learning_rate=λ)
    # loss : binary_crossentropy ; mean_squared_error ; mean_absolute_error

    model.summary()
    return model

model = Sequential()
model.add(Conv2D(32, kernel_size=5, activation='relu', input_shape=X[0].shape, padding ="same"))
model.add(Conv2D(256, kernel_size=5, activation='relu', padding ="same"))
model.add(Conv2D(64, kernel_size=3, activation='relu', padding ="same"))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(82, activation='elu')) 
opt= optimizers.Adam()
model.compile(loss='mae',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 9, 9, 32)          14144     
_________________________________________________________________
batch_normalization_2 (Batch (None, 9, 9, 32)          128       
_________________________________________________________________
flatten_5 (Flatten)          (None, 2592)              0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 2592)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 648)               1680264   
_________________________________________________________________
dropout_8 (Dropout)          (None, 648)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 324)              

In [8]:
# hyperparamètres
epochs = 50
batch_size = 64

# training
print("Training ...")
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test))

Training ...
Epoch 1/50
248/248 [==============================] - 4s 16ms/step - loss: 0.2460 - val_loss: 0.2663
Epoch 2/50
248/248 [==============================] - 4s 16ms/step - loss: 0.2160 - val_loss: 0.2489
Epoch 3/50
248/248 [==============================] - 4s 16ms/step - loss: 0.2128 - val_loss: 0.2255
Epoch 4/50
248/248 [==============================] - 4s 17ms/step - loss: 0.2109 - val_loss: 0.2203
Epoch 5/50
248/248 [==============================] - 4s 17ms/step - loss: 0.2098 - val_loss: 0.2273
Epoch 6/50
248/248 [==============================] - 4s 16ms/step - loss: 0.2087 - val_loss: 0.2218
Epoch 7/50
248/248 [==============================] - 5s 18ms/step - loss: 0.2078 - val_loss: 0.2314
Epoch 8/50
248/248 [==============================] - 5s 20ms/step - loss: 0.2070 - val_loss: 0.2161
Epoch 9/50
248/248 [==============================] - 5s 21ms/step - loss: 0.2060 - val_loss: 0.2187
Epoch 10/50
248/248 [==============================] - 5s 19ms/step - loss: 0.

In [6]:
proba = model.predict(X_train)

In [7]:
print(proba[5])

[0.5228609  0.55699044 0.5362773  0.48673797 0.5731567  0.58022106
 0.44858414 0.53510237 0.44719613 0.58502877 0.55413914 0.5147674
 0.5453913  0.5646487  0.49700922 0.5359767  0.52565235 0.6096695
 0.5047757  0.5869566  0.5456616  0.5773529  0.5962179  0.49978694
 0.5577977  0.4602521  0.48288396 0.5676175  0.52821857 0.5870718
 0.60353786 0.46679008 0.52251834 0.4387975  0.44744998 0.5341391
 0.5221214  0.41564643 0.5767388  0.5387567  0.5859307  0.5048639
 0.5151499  0.5288731  0.55381525 0.46310264 0.531793   0.57807016
 0.5512063  0.5834061  0.5396954  0.52075064 0.5956475  0.5469848
 0.5789376  0.6177249  0.50771105 0.6271464  0.4595841  0.5756392
 0.5680325  0.5585363  0.46658286 0.57207024 0.5117404  0.51146805
 0.5826603  0.50609857 0.61082166 0.5373887  0.5735155  0.5950719
 0.5076919  0.5194654  0.50452113 0.58208615 0.5051     0.625646
 0.6499548  0.5460365  0.5223807  0.47065794]
